# Young Star Magnetic Models

Convergence of magnetic models is improved with a [new treatment of the peak magnetic field strength](http://nbviewer.ipython.org/github/gfeiden/Notebook/blob/master/Daily/20150728_peak_magnetic_field.ipynb) definition. Previously, it was defined as either $R_{\rm peak} = 0.5R_{\star}$ or $R_{\rm peak} = R_{\rm tach}$, where $R_{\rm tach}$ is the radial location of the interface region between the stellar radiation and convection zones (i.e., the tachocline). This caused problems for young star models as models start off fully convective but develop radiative cores as the central temperature increases throughout its gravitational contraction. Magnetic fields therefore jumped rapidly from a fully convective treatment to a partially convective treatment, leading to excessively large interior magnetic field strengths. To avoid this problem, the peak magnetic field strength is treated as either $R_{\rm peak} = 0.5R_{\star}$ or $R_{\rm peak} = R_{\rm tach}$, _whichever is larger_, in all cases.

Two small grids of magnetic models are computed with GS98 and GAS07 solar abundances. These may be incorporated into the Young Star manuscript, where we present models of young stars that have now been used in several publications (e.g., [Malo et al. 2014](http://adsabs.harvard.edu/abs/2014arXiv1406.6750M); [Herczeg & Hillenbrand 2015](http://adsabs.harvard.edu/abs/2015arXiv150506518H)). However, these models are computed, specifically, at the request of I. Song, who wishes to incorporate magnetic models into an analysis. The tracks themselves will not be incorporated into the GitHub repo, as publishing the full grid would require too much disk space, but they are available [upon request](mailto:gregory.a.feiden@gmail.com?subject=Magnetic model request).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Magnetic Mass Tracks

We'll start with loading mass tracks from the GAS07 solar abundance subset. These adopt surface boundary conditions from the MARCS model atmosphere structures. While we typically recommend surface boundary conditions be attached at an optical depth where $\tau_{\rm ross} \ge 50$, the magnetic models are computed by fitting surface boundary conditions where $\tau_{\rm ross} \ge 10$. Magnetic fields largely affect the super-adiabiatic layers near the stellar surface, with deeper field strenghts playing a less critical role ([Feiden & Chaboyer 2013](http://adsabs.harvard.edu/abs/2013ApJ...779..183F), [2014](http://adsabs.harvard.edu/abs/2014A%26A...571A..70F)). However, the motivation for attaching the boundary conditions at larger optical depths is to provide a better treatment of super-adiabatic layers where radiation and convection are both significnat contributors to the total energy flux ([Chabrier & Baraffe 1997](http://adsabs.harvard.edu/abs/1997A%26A...327.1039C)), which is in opposition to our efforts of including the effects of magnetic fields. 

We provide a compromise by fixing the surface boundary conditions at higher layer in the star. This provides a sufficiently large super-adiabatic layer to give the magnetic field a reasonable influence, while still providing a reliable estimate of the surface conditions that help set the overall thermal structure of the star.

In [2]:
masses = np.arange(0.1, 0.96, 0.05)  # list of masses

## Magnetic Isochrones

Process the magnetic mass tracks into isochrones. Since mass tracks are computed with a relatively course mass resolution ($0.05 M_{\odot}$), spline interpolation is used to smooth the resulting isochrones with a finer mass resolution. 

In [5]:
from scipy.interpolate import interp1d

ages = np.arange(5.0e6, 3.1e7, 1.0e6)  # ages requested

In [18]:
# open output file objects 
output_files = [open('files/dmestar_{:07.1f}myr_gas07_z+0.00_a+0.00_mag25kG.iso'.format(age/1.0e6), 'w') 
                for age in ages]

trk_directory = '../../evolve/dmestar/trk/gas07/p000/a0/amlt2040/mag25kG'
for mass in masses:
    trk_filename = 'm{:04.0f}_GAS07_p000_p0_y26_mlt2.040_mag25kG.trk'.format(mass*1000.)
    try:
        gas07_trk = np.genfromtxt('{0}/{1}'.format(trk_directory, trk_filename), usecols=(0, 1, 2, 3, 4, 8))
    except IOError:
        continue
    
    # extract only relevant age chunk for easier interpolation
    gas07_trk = np.array([time_step for time_step in gas07_trk if 1.0e6 <= time_step[0] <= 5.0e7])
    
    # generate linear interpolation curve as a function of age
    try:
        icurve = interp1d(gas07_trk[:, 0], gas07_trk[:, 1:], kind='linear', axis=0)
    except IndexError:
        continue
    
    # extract properties at the requested age
    trk_props = icurve(ages)
    
    i = 0
    for props in trk_props:
        s = '{:6.3f}'.format(mass)
        for prop in props:
            s += '{:14.6f}'.format(prop)
        s += '\n'
        output_files[i].write(s)
        i += 1
    #print "{:4.2f} Mo Track Processed.".format(mass)
    
# close output files
for f in output_files:
    f.close()

Interpolate isochrones onto a finer mass grid.

In [19]:
fine_mass_grid = np.arange(0.1, 0.95, 0.02)
for age in ages:
    iso_filename = 'files/dmestar_{:07.1f}myr_gas07_z+0.00_a+0.00_mag25kG.iso'.format(age/1.0e6)
    isochrone = np.genfromtxt(iso_filename)
    
    # generate interpolation curves
    icurve = interp1d(isochrone[:,0], isochrone, axis=0, kind='cubic')
    
    # interpolate onto a finer mass grid
    fine_isochrone = icurve(fine_mass_grid)
    
    # write header
    header  = 'Dartmouth Stellar Evolution Model: Quick Isochrone \n\n'
    header += 'Age = {:7.1f} Myr   [Fe/H] = {:+5.2f}   [a/Fe] = {:+5.2f} \n\n'.format(age/1.e6, 0.0, 0.0)
    header += '{:^14} {:^14} {:^14} {:^14} {:^14} {:^14}'.format('Mass', 'log(Teff)', 'log(g)', 'log(L/Lo)', 
                                                                 'log(R/Ro)', 'A(Li)')
    
    # overwrite original file
    np.savetxt(iso_filename, fine_isochrone, fmt='%14.6f', header=header)

Magnetic isochrones are stored in the directory `files/` and follow the format outline in the two code snippets above. We can take a quick look at some of the proeprties of these isochrones and how they compare to standard stellar evolution isochrones (i.e., without a magnetic perturbation).